In [865]:
import numpy as np
import itertools
import matplotlib.pyplot as plt

## Kannan's SVP

In [866]:
# Ортогонализация Грама-Шмидта. Возвращает ортогонализированный базис и коэффициенты мю
# В случае если матрица на входе полного ранга, то это будет QR разложение, где столбцы Q попарно ортогональны, 
# а R - верхнетреугольная. (проверено ниже)
def GSO(basis):
    n = basis.shape[1]
    dim = basis.shape[0]
    new_basis = np.array(basis, copy=True)
    R = np.identity(dim)[:,:n]
    for j in range(1, n):
        for i in range (0, j):
            R[i,j] = np.dot(basis[:,j],new_basis[:,i])/np.dot(new_basis[:,i],new_basis[:,i])
            new_basis[:,j]-=R[i,j]*new_basis[:,i]
    return new_basis, R

In [867]:
a = np.array([[1,-1,3],[1,0.0,5],[1,2,6]])

In [868]:
a

array([[ 1., -1.,  3.],
       [ 1.,  0.,  5.],
       [ 1.,  2.,  6.]])

In [869]:
b,r = GSO(a)

In [870]:
b,r

(array([[ 1.        , -1.33333333, -0.42857143],
        [ 1.        , -0.33333333,  0.64285714],
        [ 1.        ,  1.66666667, -0.21428571]]),
 array([[1.        , 0.33333333, 4.66666667],
        [0.        , 1.        , 0.92857143],
        [0.        , 0.        , 1.        ]]))

In [871]:
np.matmul(b,r)

array([[ 1., -1.,  3.],
       [ 1.,  0.,  5.],
       [ 1.,  2.,  6.]])

In [872]:
# LLL редукция. Более простая для вычисления, но не всегда даёт лучший базис.
# Используется как первый шаг для облегчения дальнейшей KZ редукции
def LLL(B, delta):
    B_new, R = GSO(B)
    B_red = np.array(B, copy=True)  
    n, k = B.shape[1], 1
    while k < n:
        # size reduction step
        for j in reversed(range(k)):
            if abs(R[j,k]) > .5:
                B_red[:,k] = B_red[:,k] - round(R[j,k])*B_red[:,j]
                B_new, R = GSO(B_red)

        # swap step
        if np.dot(B_new[:,k],B_new[:,k]) >= (delta - R[k-1,k]**2)*np.dot(B_new[:,k-1],B_new[:,k-1]):
            k = k + 1
        else:
            B_red[:,[k,k-1]] = B_red[:,[k-1,k]]
            B_new, R = GSO(B_red)
            k = max(k-1, 1)

    return B_red 

In [873]:
# Для двумерных решёток. Используется в частичной KZ и как база рекурсии,
# Поскольку для таких решёток KZreducted == GaussReducted
def GaussReduction(b1,b2):
    b1_n = np.dot(b1,b1)
    mu = np.dot(b1,b2)/b1_n
    b2 = b2 - round(mu)*b1
    b2_n = np.dot(b2,b2)
    while b2_n < b1_n:
        temp = b2
        b2 = b1
        b1 = temp
        b1_n = b2_n
        mu = np.dot(b1,b2)/b1_n
        b2 = b2 - round(mu)*b1
        b2_n = np.dot(b2,b2)
    return(b1,b2)

In [874]:
# Нормализация
def SizeReduction(B):
    B_new, R = GSO(B)
    B_red = np.array(B, copy=True)  
    n, k = B.shape[1], 1
    while k < n:
        for j in reversed(range(k)):
            if abs(R[j,k]) > .5:
                B_red[:,k] = B_red[:,k] - round(R[j,k])*B_red[:,j]
                B_new, R = GSO(B_red)
        k = k + 1
    return B_red

In [875]:
# Проекция на ортогональное дополнение к первому вектору, по сути - внешний цикл GSO с фиксированным i = 0 
def Projections(basis):
    n = basis.shape[1]
    new_basis = np.array(basis, copy=True)
    b1_n = np.dot(new_basis[:,0],new_basis[:,0])
    for j in range(1, n):
            new_basis[:,j]-=np.dot(basis[:,j],new_basis[:,0])*new_basis[:,0]/b1_n
    return new_basis


In [876]:
# Нахождение минимального вектора на частично KZ-редуцированном базисе
def Enumeration(basis):
    new_basis, R = GSO(basis)
    dim = basis.shape[0]
    n = basis.shape[1]
    ranges = []
    for i in reversed(range(n)):
        threshold = np.floor(np.sqrt(np.dot(basis[:,0],basis[:,0])/np.dot(new_basis[:,i],new_basis[:,i]))).astype(int)
        ranges.append(range(-threshold,threshold+1))
    SV = basis[:,0]
    sv_norm = np.dot(basis[:,0],basis[:,0])
    ranges = list(itertools.product(*ranges))
    for alphas in ranges:
        v = np.matmul(basis,list(alphas))
        curnorm = np.dot(v,v)
        if curnorm < sv_norm and curnorm > 1e-15:
            print(alphas)
            sv_norm = curnorm
            SV = v             
    return SV    

In [877]:
# Частичная KZ-редукция
def PartialKZreduction(basis, delta):
    new_basis = LLL(basis, delta)
    cond = True
    while cond:
        _, kz_reduced, transf_matr = KZreduction(new_basis[:,1:], delta)
        new_basis[:,1:] = np.matmul(new_basis[:,1:],transf_matr)
        new_basis = SizeReduction(new_basis)
        if np.dot(kz_reduced[:,1],kz_reduced[:,1]) >= np.dot(new_basis[:,0],new_basis[:,0])/4:
            cond = False
        else:
            new_basis[:,0], new_basis[:,1] = GaussReduction(new_basis[:,0],new_basis[:,1])
    return new_basis

In [878]:
# Полная KZ-редукция. В случае, если размеронсть подпространства решётки = 2, вызывается гаусс-редукция
# Иначе вызывается частичная, которая в себе снова вызывает полную, но уже для базиса без первого вектора.
def KZreduction(basis, delta):
    new_basis = np.array(basis, copy=True)  
    if basis.shape[1] > 2:
        new_basis = PartialKZreduction(basis, delta)
        SV = Enumeration(new_basis)
        new_basis[:,0] = SV
        new_basis = LLL(new_basis,delta)
        _, kz_reduced, transf_matr = KZreduction(new_basis[:,1:], delta)
        new_basis[:,1:] = np.matmul(new_basis[:,1:],transf_matr)
        new_basis = SizeReduction(new_basis)
    else:
        new_basis[:,0], new_basis[:,1] = GaussReduction(basis[:,0],basis[:,1])
        SV = new_basis[:,0]
    tf_m = np.matmul(np.matmul(np.linalg.inv(np.matmul(np.transpose(basis),basis)),np.transpose(basis)),new_basis)
    return SV, new_basis, tf_m 

In [879]:
SV, new_basis, _ = KZreduction(a,0.75)

In [880]:
print(SV)

[0. 1. 0.]


In [881]:
print(new_basis)

[[ 0.  1. -1.]
 [ 1.  0.  0.]
 [ 0.  1.  2.]]


## Randomized SVP

In [882]:
# Генерация точек внутри сферы радиуса 2 с центром в начале координат. (равномерное распределение)
# Метод: https://mathworld.wolfram.com/BallPointPicking.html
def randsphere(n_points, dim):
    res = np.random.normal(0,1,(n_points,dim))
    mult = np.random.exponential(1,(n_points))
    for i in range(n_points):
        mult[i] = 2/np.sqrt((np.dot(res[i],res[i]) + mult[i]))
        res[i]*=mult[i]
    return res

In [883]:
def RandomizedSVP(basis):
    n = basis.shape[1]
    dim = basis.shape[0]
    R = 0
    norms = [np.sqrt(np.dot(basis[:,i],basis[:,i])) for i in range(n)]
    R = sum(norms)
    inv_b = np.linalg.inv(basis)
    N_samples = round(2**(8*dim)*np.log(R))
    
    X = randsphere(N_samples, dim)
    L = []
    
    # Поиск ближайших точек решётки и заполнение L
    for x in X:
        lam = np.matmul(inv_b, x)
        lam = lam - np.round(lam)
        y = np.matmul(basis, lam)
        L.append([x,y])
    L = np.array(L)
    
    # 
    while R > 6:
        J = [0]
        eta = [0]*N_samples
        for i in range(1,N_samples):
            eta[i] = i
            j = 0
            # Просеивание с поиском центров
            while j < len(J) and eta[i] == i:
                v = L[i,1] - L[J[j],1]
                if (sqrt(np.dot(v,v)) <= R/2):
                    eta[i] = J[j]
                j = j + 1 
            if eta[i] == i:
                J.append(i)
                
        # Удаляем центры из L
        L = np.delete(L, J, 0)
        N_samples = N_samples - len(J)
        
        # Замена всего, что осталось
        for i in range(N_samples):
            L[i] = (L[i,0],L[i,1] - (L[eta[i],1] - L[eta[i],0]))
        R = R/2 + 2
    # Поиск минимума среди попарных разностей
    res = []
    min_norm = dim*sum(norms)
    print('Количество итераций: ', N_samples)
    for i in range(N_samples - 1):
        print(res)
        for j in range(i + 1, N_samples):
            v = (L[i,1] - L[i,0]) - (L[j,1] - L[j,0])
            cur_norm = np.dot(v,v)
            if cur_norm < min_norm and cur_norm > 1e-15:
                min_norm = cur_norm
                res = v
    return v

In [884]:
ttt = np.array([[1,2.0],[2,-3]])

In [885]:
# Длина минимального вектора лежит в [2,3)
b = LLL(ttt,0.75)
print(np.sqrt(np.dot(b[:,0],b[:,0])))

2.23606797749979


In [886]:
RandomizedSVP(ttt)

Количество итераций:  115672
[]
[1. 2.]
[1. 2.]
[1. 2.]
[1. 2.]
[1. 2.]


KeyboardInterrupt: 